## Exploration des données de réponses à questions ouvertes

In [ ]:
import pandas as pd
from text_cleaning import clean_df_col, get_tokens_from_doc

In [ ]:
DATA_FOLDER = "../Data/"

In [ ]:
filename = "liste_des_reponses_aux_questions_ouvertes_par_consultation_2023-11-02T10_21_31.386925747Z.csv"

In [ ]:
filename = "liste_des_reponses_aux_questions_ouvertes_par_consultation_2023-11-02T10_21_31.386925747Z.csv"
filename_with_demo = "reponses_aux_questions_ouvertes_croisees_avec_les_donnees_demo_2023-11-02T13_09_36.480435917Z.csv"
df = pd.read_csv(DATA_FOLDER + filename)
df_demo = pd.read_csv(DATA_FOLDER + filename_with_demo)

In [ ]:
QUESTION_COL = "Questions → Title"
RESPONSE_COL = "Response Text"

In [ ]:
# Queqstions existantes
df[QUESTION_COL].unique()

In [ ]:
df

In [ ]:
df_filtered = df[df[QUESTION_COL] == "Quelles sont vos autres propositions pour lutter contre les violences faites aux enfants ?"].copy()
df_filtered

### Text preprocessing

In [ ]:
#TODO: Load NLP à part et le passer en argument pour augmenter les perfs
clean_df_col(df, RESPONSE_COL, "token_col")
df

In [ ]:
X = df_filtered[RESPONSE_COL]

Cleaning text

In [ ]:
from text_cleaning import text_process_nltk

### Vectorizing 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfid = TfidfVectorizer(analyzer=text_process_nltk, ngram_range=(1,3)).fit(X)

In [ ]:
print(len(tfid.vocabulary_))
X_transformed = tfid.transform(X)

### Clustering

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
sum_of_squared_distances = []
K = range(1, 10)
for k in K:
    km = KMeans(n_clusters=k, init='k-means++', n_init=100)
    km = km.fit(X_transformed)
    sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum of squared distances')
plt.title("Elbow method for optimal K")
plt.show()

Evaluating the clusters

In [ ]:
k = 60
km = KMeans(n_clusters=3)
km = km.fit(X_transformed)

In [ ]:
Y = km.transform(X_transformed)

In [ ]:
Y

In [ ]:
data = {"response": X,
        "cluster": Y}
results = pd.DataFrame({"response": X})
results["cluster"] = Y

In [ ]:
X_transformed

In [ ]:
# Get Synthetised answer from each cluster
# Maybe use claud for this, ask Penelope 